## Redução de dimensionalidade

## Importação de bibliotecas

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, metrics
from sklearn.neural_network import BernoulliRBM
from sklearn.model_selection import train_test_split

# Importação da classe para rede neural utilizando o scikit-learn
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

## Criação da Base de Dados

In [2]:
digits = datasets.load_digits()
previsores = np.asarray(digits.data, 'float32')
classe = digits.target

## Criação do normalizador - Escalonamento

In [3]:
normalizador = MinMaxScaler(feature_range = (0,1))
previsores = normalizador.fit_transform(previsores)

## Divisão da base de treino e teste

In [4]:
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size=0.2, random_state=0)

## Criar estrutura do RBM - Restricted Boltzmann Machine

In [5]:
# Criação e configuração da 
rbm = BernoulliRBM(random_state=0)
rbm.n_iter = 25
rbm.n_components = 50

## Criação da rede neural usando scikit-learn

In [7]:
# O parâmetro hidden_layer_sizes cria as camadas escondidas, sendo que cada número
# 37 representa uma camada. Neste exemplo temos duas camadas escondidas com 37 
# neurônios cada uma - usada a fórmula (entradas + saídas) / 2 = (64 + 10) / 2 = 37
# No scikit-learn não é necessário configurar a camada de saída, pois ele 
# faz automaticamente. Definimos o max_iter com no máximo 1000 épocas, porém,
# quando a loos function não melhora depois de um certo número de rodadas ele
# pára a execução. O parâmetro verbosa mostra as mensagens na tela
mlp_rbm = MLPClassifier(hidden_layer_sizes = (37, 37),
                        activation = 'relu', 
                        solver = 'adam',
                        batch_size = 50,
                        max_iter = 1000,
                        verbose = 1)

## Criação do pipeline para executarmos o rbm e logo após o mlp

In [8]:
classificador_rbm = Pipeline(steps=[('rbm', rbm), ('mlp', mlp_rbm)])
classificador_rbm.fit(previsores_treinamento, classe_treinamento)

Iteration 1, loss = 2.23863865
Iteration 2, loss = 2.01292421
Iteration 3, loss = 1.69923007
Iteration 4, loss = 1.31352058
Iteration 5, loss = 0.95474917
Iteration 6, loss = 0.72331949
Iteration 7, loss = 0.58516827
Iteration 8, loss = 0.50221604
Iteration 9, loss = 0.44309032
Iteration 10, loss = 0.40339132
Iteration 11, loss = 0.37434827
Iteration 12, loss = 0.34796821
Iteration 13, loss = 0.33193947
Iteration 14, loss = 0.31526825
Iteration 15, loss = 0.29967217
Iteration 16, loss = 0.29112591
Iteration 17, loss = 0.28305146
Iteration 18, loss = 0.26986669
Iteration 19, loss = 0.26312496
Iteration 20, loss = 0.25597223
Iteration 21, loss = 0.25233099
Iteration 22, loss = 0.24381311
Iteration 23, loss = 0.24004308
Iteration 24, loss = 0.23468972
Iteration 25, loss = 0.23061009
Iteration 26, loss = 0.22707451
Iteration 27, loss = 0.21964472
Iteration 28, loss = 0.21926045
Iteration 29, loss = 0.21392808
Iteration 30, loss = 0.20940013
Iteration 31, loss = 0.20990261
Iteration 32, los

Pipeline(steps=[('rbm',
                 BernoulliRBM(n_components=50, n_iter=25, random_state=0)),
                ('mlp',
                 MLPClassifier(batch_size=50, hidden_layer_sizes=(37, 37),
                               max_iter=1000, verbose=1))])

### Previsões usando rbm + mlp

In [11]:
previsoes_rbm = classificador_rbm.predict(previsores_teste)
precisao_rbm = metrics.accuracy_score(previsoes_rbm, classe_teste)
precisao_rbm

0.95

## Criação da rede neural simples sem aplicação de rbm

In [12]:
mlp_simples = MLPClassifier(hidden_layer_sizes = (37, 37),
                        activation = 'relu', 
                        solver = 'adam',
                        batch_size = 50,
                        max_iter = 1000,
                        verbose = 1)
mlp_simples.fit(previsores_treinamento, classe_treinamento)
previsoes_mlp = mlp_simples.predict(previsores_teste)
precisao_mlp = metrics.accuracy_score(previsoes_mlp, classe_teste)
precisao_mlp

Iteration 1, loss = 2.29709207
Iteration 2, loss = 1.98955132
Iteration 3, loss = 1.60978386
Iteration 4, loss = 1.15693512
Iteration 5, loss = 0.79751130
Iteration 6, loss = 0.57133705
Iteration 7, loss = 0.43896326
Iteration 8, loss = 0.35832610
Iteration 9, loss = 0.29956323
Iteration 10, loss = 0.26162296
Iteration 11, loss = 0.23273030
Iteration 12, loss = 0.20645656
Iteration 13, loss = 0.19299108
Iteration 14, loss = 0.17561953
Iteration 15, loss = 0.16187304
Iteration 16, loss = 0.15268833
Iteration 17, loss = 0.14271888
Iteration 18, loss = 0.13595041
Iteration 19, loss = 0.12716892
Iteration 20, loss = 0.11682642
Iteration 21, loss = 0.11276603
Iteration 22, loss = 0.10745614
Iteration 23, loss = 0.10056440
Iteration 24, loss = 0.09665030
Iteration 25, loss = 0.09210748
Iteration 26, loss = 0.09147813
Iteration 27, loss = 0.08462994
Iteration 28, loss = 0.08060215
Iteration 29, loss = 0.08008596
Iteration 30, loss = 0.07469682
Iteration 31, loss = 0.07231216
Iteration 32, los

0.9805555555555555

In [14]:
# Comparando os resultados, com RBM chegamos em 0.93 e sem RBM o percentual é de 0.98
# Com isso chegamos a conclusão que usar RBM com essa base de dados e com redes
# neurais piora os resultados